# ***Abstract :***
# This project is built to detect the depressive based on the tweet content user posted. 

***Recommend using colab to run my program***

In [1]:
# pip and import necessary file and library

! git clone https://github.com/Jingxuan-Bao/Tweet_Depression_Detection.git
! pip install contractions
! pip install joblib
! pip install files
! pip install transformers

import pandas as pd
import re
import string
import nltk
import contractions
import numpy as np
nltk.download('stopwords')


from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from google.colab import files
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score

import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

Cloning into 'Tweet_Depression_Detection'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 104 (delta 28), reused 85 (delta 14), pack-reused 0
Receiving objects: 100% (104/104), 27.12 MiB | 4.83 MiB/s, done.
Resolving deltas: 100% (28/28), done.
Updating files: 100% (34/34), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for files: filename=files-1.1.1-py3-none-any.whl size=3668 sha256=1cb25eabf0e44f8a8325904570c0ebe

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Section 1: Load Data**

In [2]:
# read normal tweet data from my github

normal_tweet = pd.read_csv('/content/Tweet_Depression_Detection/dataset/normal_tweet.csv')
print(normal_tweet.head())

                                                text  label
0  in class. im EARLY. bummed RYAN SEACREST never...      0
1                                   i wanna go home       0
2                                  Our AC is broken       0
3  feels that the only place where Federer (&amp;...      0
4  @tommcfly Yeah tom, you are always the one tha...      0


In [3]:
normal_tweet

,text,label
0,in class. im EARLY. bummed RYAN SEACREST never...,0
1,i wanna go home,0
2,Our AC is broken,0
3,feels that the only place where Federer (&amp;...,0
4,"@tommcfly Yeah tom, you are always the one tha...",0
...,...,...
59995,It's 8:40am... I've been awake for nearly 2 ho...,0
59996,ughh mom`s making me cook dinner when i`m in t...,0
59997,"argh. netregistry is pissing me off today, pos...",0
59998,@gemmak500 It's amazing how many couples you s...,0


In [4]:
# read depressive tweet data from my github

depressive_tweet = pd.read_csv('/content/Tweet_Depression_Detection/dataset/depressive_tweet.csv')
print(depressive_tweet.head())

                                               Tweet
0  “Our world today so desperately hungers for ho...
1  “Go if you have to, but remember, don’t come b...
2  #Artists without an air of #loneliness , are #...
3  Getting a bit tired of losing my mind \n\n#bip...
4  Maturity is when you realize\nYour image, rela...


In [5]:
depressive_tweet

,Tweet
0,“Our world today so desperately hungers for ho...
1,"“Go if you have to, but remember, don’t come b..."
2,"#Artists without an air of #loneliness , are #..."
3,Getting a bit tired of losing my mind \n\n#bip...
4,"Maturity is when you realize\nYour image, rela..."
...,...
4525,Chad is upset. Why'd you not wash your hands? ...
4526,I have never learnt the art of learning!\n#Sad
4527,Just got banned from a server F #sad
4528,I literally cried during my exam and the cam i...


In [6]:
# preprocess the depressive tweet, extract the target content, clean null data

depressive_tweet['label'] = 1
depressive_tweet.rename(columns= {"Tweet" : "text"}, inplace=True)
depressive_tweet = depressive_tweet[['text', 'label']]
depressive_tweet = depressive_tweet.dropna()

In [7]:
depressive_tweet

,text,label
0,“Our world today so desperately hungers for ho...,1
1,"“Go if you have to, but remember, don’t come b...",1
2,"#Artists without an air of #loneliness , are #...",1
3,Getting a bit tired of losing my mind \n\n#bip...,1
4,"Maturity is when you realize\nYour image, rela...",1
...,...,...
4525,Chad is upset. Why'd you not wash your hands? ...,1
4526,I have never learnt the art of learning!\n#Sad,1
4527,Just got banned from a server F #sad,1
4528,I literally cried during my exam and the cam i...,1


In [8]:
depressive_tweet = depressive_tweet.sample(n=4000, random_state=42)
normal_tweet = normal_tweet.sample(n=6000, random_state=42)

# **Section 2: Data Processing**

In [9]:
# merge normal tweet and depressive tweet

df = pd.concat([normal_tweet,depressive_tweet])
# shuffle the data
df = df.sample(frac=1)

In [10]:
df

,text,label
2755,this is my last year laughing at the psat meme...,1
2196,Being manly isn't wearing the mask and ignorin...,1
5763,I hate soar throats,0
11711,@JoelMadden i never feel earthquakes either..o...,0
4291,skz flop #dipriz #sad #wap #brocken #nickiminaj,1
...,...,...
58602,@tehroxie ah yush they would And if they woul...,0
50709,is taking Alix Eve to LGA for her flight to CM...,0
39582,Laptop keeps freezing and I can't do this artw...,0
50039,Goes back to school today... its not looking f...,0


In [11]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_tweet_text(tweet_text):
    # Conver to string 
    tweet_text = str(tweet_text)

    # Convert to lowercase
    tweet_text = tweet_text.lower()

    # Expand contractions
    tweet_text = contractions.fix(tweet_text)

    # Remove URLs
    tweet_text = re.sub(r"http\S+", "", tweet_text)

    # Remove mentions (@) and hashtags (#)
    tweet_text = re.sub(r'\@\w+|\#', '', tweet_text)

    # Remove emojis
    tweet_text = remove_emojis(tweet_text)

    # Remove punctuation
    tweet_text = tweet_text.translate(str.maketrans("", "", string.punctuation))

    # Remove numbers
    tweet_text = re.sub(r"\d+", "", tweet_text)

    # Remove whitespace
    tweet_text = tweet_text.strip()

    # Tokenize the text into individual words
    tweet_words = tweet_text.split()

    # Remove stop words using NLTK library
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tweet_words = [word for word in tweet_words if word not in stop_words]

    # Join the remaining words back into a string
    processed_tweet_text = " ".join(tweet_words)

    return processed_tweet_text

In [12]:
df['text'] = df['text'].apply(preprocess_tweet_text)

In [13]:
df

,text,label
2755,last year laughing psat memes sad,1
2196,manly wearing mask ignoring depression manly t...,1
5763,hate soar throats,0
11711,never feel earthquakes eitherone time bel air ...,0
4291,skz flop dipriz sad wap brocken nickiminaj,1
...,...,...
58602,ah yush would would taken race could angstclub...,0
50709,taking alix eve lga flight cmh overnight visit...,0
39582,laptop keeps freezing cannot artwork need samp...,0
50039,goes back school today looking forward extreme...,0


In [14]:
df = df.sample(n=5000, random_state=42)

# **Section 3: Pre Trained Model - BERT**

In [15]:
# Load dataset
data = df
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Set up tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize data
train_encodings = tokenizer(train_data['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_data['text'].tolist(), truncation=True, padding=True, max_length=128)

# Create custom Dataset class
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_data['label'].tolist())
val_dataset = TweetDataset(val_encodings, val_data['label'].tolist())

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    evaluation_strategy="steps"
)

# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train model
trainer.train()
eval_results = trainer.evaluate()

# Generate classification repoty
val_predictions = trainer.predict(val_dataset).label_ids
val_labels = val_data['label'].tolist()
print(classification_report(val_labels, val_predictions))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss
100,0.191600,0.065499
200,0.099300,0.069072
300,0.067800,0.074637
400,0.058400,0.092610
500,0.052900,0.083515
600,0.026600,0.089831
700,0.034500,0.064251


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       593
           1       1.00      1.00      1.00       407

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

